In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("../dataset/epl-2024-2025-data.csv")

In [5]:
df.isnull().sum()

Div         0
Date        0
Time        0
HomeTeam    0
AwayTeam    0
FTHG        0
FTAG        0
FTR         0
HTHG        0
HTAG        0
HTR         0
Referee     0
HS          0
AS          0
HST         0
AST         0
HF          0
AF          0
HC          0
AC          0
HY          0
AY          0
HR          0
AR          0
dtype: int64

In [6]:
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
print("\nNumeric summary:\n", df[numeric_cols].describe())

cat_cols = df.select_dtypes(include=['object', 'category']).columns
for col in cat_cols:
    print(f"\nValue counts for {col}:\n", df[col].value_counts().head(5))



Numeric summary:
              FTHG        FTAG        HTHG        HTAG          HS          AS  \
count  380.000000  380.000000  380.000000  380.000000  380.000000  380.000000   
mean     1.513158    1.421053    0.752632    0.610526   13.752632   12.165789   
std      1.277917    1.189922    0.875867    0.828831    5.588045    5.409987   
min      0.000000    0.000000    0.000000    0.000000    2.000000    1.000000   
25%      1.000000    1.000000    0.000000    0.000000   10.000000    9.000000   
50%      1.000000    1.000000    1.000000    0.000000   13.000000   11.500000   
75%      2.000000    2.000000    1.000000    1.000000   17.000000   15.000000   
max      7.000000    6.000000    4.000000    5.000000   36.000000   37.000000   

              HST         AST          HF          AF          HC          AC  \
count  380.000000  380.000000  380.000000  380.000000  380.000000  380.000000   
mean     4.834211    4.265789   10.789474   11.276316    5.426316    4.871053   
std     

In [ ]:
plt.figure(figsize=(12,8))


<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>